In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, precision_recall_curve

from scripts import featurizer_xgb

import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier


%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../data/dfff.csv", index_col=False)

In [3]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,0.000518,0.003183,0.005264,0.004306,0.002719,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0
1,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,0.001186,1.000928,0.005655,0.007953,0.095926,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0
2,0.392041,0.005770,0.029682,0.810262,0.006672,0.180378,0.005028,NaN,0.126794,0.210560,...,0.006967,NaN,NaN,NaN,NaN,4488928018c20bf6261e69c8a94a47b6206a2fe91480b8...,2017-07-29,CR,NaN,1
3,0.673978,0.588604,0.295201,0.024650,0.007951,0.125583,0.008760,NaN,0.005867,0.175950,...,0.000805,0.003558,0.008268,0.003020,0.002798,9296e983c3e4f99b046952397fc4ba950abf1032a9bd6c...,2017-03-30,CO,R,1
4,0.778075,0.006930,0.056721,0.819369,0.004922,0.089214,0.006884,0.041515,0.002841,0.001179,...,0.009053,0.004670,0.000125,0.000819,0.000342,3fa10314686e0e51d8f76235eceab91d70aa3f24a98c3e...,2018-01-17,CL,O,0


In [4]:
df['S_2'] = pd.to_datetime(df['S_2'])

In [5]:
df['DayOfWeek'] = df['S_2'].dt.isocalendar().week.astype("int64")

In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276572 entries, 0 to 276571
Data columns (total 124 columns):
 #    Column       Dtype         
---   ------       -----         
 0    P_2          float64       
 1    D_39         float64       
 2    B_1          float64       
 3    B_2          float64       
 4    R_1          float64       
 5    S_3          float64       
 6    D_41         float64       
 7    D_43         float64       
 8    D_44         float64       
 9    B_4          float64       
 10   D_45         float64       
 11   B_5          float64       
 12   R_2          float64       
 13   D_46         float64       
 14   D_47         float64       
 15   B_6          float64       
 16   B_7          float64       
 17   D_50         float64       
 18   D_51         float64       
 19   B_9          float64       
 20   R_3          float64       
 21   D_52         float64       
 22   P_3          float64       
 23   B_10         float64       
 24 

In [7]:
df['week'] = df['S_2'].dt.week

/home/braian/Documents/Facultad/OrgaDatos/tp3-trabajo-final/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [8]:
df['S_2'].dt.day

0         10
1          4
2         29
3         30
4         17
          ..
276567    10
276568    29
276569    23
276570    30
276571     3
Name: S_2, Length: 276572, dtype: int64

In [9]:
df['DayOfYear'] = df['S_2'].dt.dayofyear

In [10]:
df

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,D_144,D_145,customer_ID,S_2,D_63,D_64,target,DayOfWeek,week,DayOfYear
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,0.004306,0.002719,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0,32,32,222
1,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,0.007953,0.095926,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0,27,27,185
2,0.392041,0.005770,0.029682,0.810262,0.006672,0.180378,0.005028,NaN,0.126794,0.210560,...,NaN,NaN,4488928018c20bf6261e69c8a94a47b6206a2fe91480b8...,2017-07-29,CR,NaN,1,30,30,210
3,0.673978,0.588604,0.295201,0.024650,0.007951,0.125583,0.008760,NaN,0.005867,0.175950,...,0.003020,0.002798,9296e983c3e4f99b046952397fc4ba950abf1032a9bd6c...,2017-03-30,CO,R,1,13,13,89
4,0.778075,0.006930,0.056721,0.819369,0.004922,0.089214,0.006884,0.041515,0.002841,0.001179,...,0.000819,0.000342,3fa10314686e0e51d8f76235eceab91d70aa3f24a98c3e...,2018-01-17,CL,O,0,3,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276567,0.193743,0.533914,0.140841,0.126221,0.500161,0.158571,0.165877,NaN,NaN,0.340831,...,0.150231,0.276447,288144823325d823e5f8a47dda65d2e0ed09a9f78bebcb...,2017-03-10,CR,U,1,10,10,69
276568,0.402107,0.914917,0.442733,0.000432,0.003491,NaN,0.436728,0.291674,0.131315,0.474846,...,0.008715,0.005995,0c3757cb750959c88f6edf19609e7195ca8ff2b6f72a30...,2017-09-29,CO,R,0,39,39,272
276569,0.841845,0.004404,0.010854,1.001861,0.004759,0.163909,0.007592,NaN,0.009742,0.005938,...,0.000816,0.006528,fbd8d187c9f8a05016512f66707160b600434293b06ecf...,2017-09-23,CO,O,0,38,38,266
276570,0.393324,0.117837,0.205245,0.015611,0.004225,0.564817,0.008934,0.247497,0.878193,0.533835,...,0.004125,0.009582,6a1c7ee2ef483998bd91b42107b18b84ab3012f57e9f81...,2017-04-30,CO,NaN,1,17,17,120


In [11]:
(df.customer_ID.value_counts().reset_index()['customer_ID']).astype('Int64')

0         7
1         6
2         6
3         6
4         6
         ..
208761    1
208762    1
208763    1
208764    1
208765    1
Name: customer_ID, Length: 208766, dtype: Int64

In [12]:
a = df.customer_ID.value_counts().reset_index()
#a['repeat_customer_id'] = 1 if [(df.customer_ID.value_counts().reset_index()['customer_ID']).astype('Int64') > 3] else 0

In [13]:
df.customer_ID.value_counts().reset_index(name = 'customer_id_repeat_size').rename(columns = {'index':'customer_ID'})

,customer_ID,customer_id_repeat_size
0,a19c49b64f5eea11b318c4c61bcc151d93c48bf8baa782...,7
1,5e08357268f0bf1066130b754f5c850a9ce83d5833b4ef...,6
2,46136b7c01033ac559add451862260a1765272cb3dbd88...,6
3,5382886bb9ce7018ffb123725def43a70e4ec27d767794...,6
4,1a640ff8c9ec46fafeace5a2d9ddef2244425321269a94...,6
...,...,...
208761,442b6686258a0582b16af718824cf5808dc466d73d0e87...,1
208762,3c41319b583b42fd8285a88138f096ce610879690cdad2...,1
208763,15c88923e0f4767dbe86be8fe4c96f30024f4c9f2c7e3a...,1
208764,39c432c0380f444407625dd20aa3ba3084e16e1477657f...,1


In [14]:
aux_df = df.customer_ID.value_counts().reset_index(name = 'customer_id_repeat_size').rename(columns = {'index':'customer_ID'})


In [15]:
aux_df['repeat_customer_id'] = np.where(aux_df['customer_id_repeat_size'] > 1, 1, 0)
aux_df

,customer_ID,customer_id_repeat_size,repeat_customer_id
0,a19c49b64f5eea11b318c4c61bcc151d93c48bf8baa782...,7,1
1,5e08357268f0bf1066130b754f5c850a9ce83d5833b4ef...,6,1
2,46136b7c01033ac559add451862260a1765272cb3dbd88...,6,1
3,5382886bb9ce7018ffb123725def43a70e4ec27d767794...,6,1
4,1a640ff8c9ec46fafeace5a2d9ddef2244425321269a94...,6,1
...,...,...,...
208761,442b6686258a0582b16af718824cf5808dc466d73d0e87...,1,0
208762,3c41319b583b42fd8285a88138f096ce610879690cdad2...,1,0
208763,15c88923e0f4767dbe86be8fe4c96f30024f4c9f2c7e3a...,1,0
208764,39c432c0380f444407625dd20aa3ba3084e16e1477657f...,1,0


In [16]:
df = df.merge(aux_df, how='inner', on='customer_ID').rename(columns={'customer_id_repeat_size_y': 'customer_id_repeat_size'})

In [17]:
df

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,customer_ID,S_2,D_63,D_64,target,DayOfWeek,week,DayOfYear,customer_id_repeat_size,repeat_customer_id
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0,32,32,222,2,1
1,0.915066,0.004296,0.053482,0.813882,0.005924,0.170043,0.001882,NaN,0.005569,0.185094,...,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-10-01,CO,O,0,39,39,274,2,1
2,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0,27,27,185,3,1
3,0.753555,0.006698,0.007812,0.816977,0.007578,0.156670,0.007471,0.127711,0.132021,0.046112,...,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-06-20,CO,O,0,25,25,171,3,1
4,0.778058,0.001282,0.007885,0.810770,0.001783,NaN,0.005630,0.222387,0.001884,0.019340,...,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-09-20,CO,O,0,38,38,263,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276567,0.252494,0.329432,0.165080,0.028412,0.507827,0.164967,0.006664,NaN,0.754016,0.784514,...,a8b55dbdaca102187019671ec50a1bf5d7038c43a4ef5f...,2017-07-18,CO,U,1,29,29,199,1,0
276568,0.642230,0.619251,0.036341,1.004678,0.008373,0.116031,0.004885,NaN,0.005760,0.019112,...,ae8b6c0b72c448c88463d8a4fb530bd0e756fb008c0d90...,2017-09-06,CO,R,0,36,36,249,1,0
276569,0.450034,0.032258,0.441105,0.032592,0.504759,0.161592,0.007296,0.174857,0.379909,1.031679,...,74e92c09a48f33cae5b99c3a1b997dc840b87f14a5669a...,2018-01-16,XZ,U,1,3,3,16,1,0
276570,0.423323,0.650394,0.213818,0.117303,0.006089,0.112029,0.002316,0.014977,0.751193,0.722948,...,284ee72ae535b3ce94dc48e9830200847e22778b93c00c...,2017-07-18,CO,O,0,29,29,199,1,0


In [18]:
df_featurizado = featurizer_xgb.featurizer(df)

In [19]:
'DayOfYear' in df_featurizado.columns

True

In [20]:
df_featurizado

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,D_64_O,D_64_R,D_64_U,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,customer_id_repeat_size
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,1,0,0,0,1,0,0,0,0,2
1,0.915066,0.004296,0.053482,0.813882,0.005924,0.170043,0.001882,NaN,0.005569,0.185094,...,1,0,0,0,1,0,0,0,0,2
2,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,1,0,0,0,1,0,0,0,0,3
3,0.753555,0.006698,0.007812,0.816977,0.007578,0.156670,0.007471,0.127711,0.132021,0.046112,...,1,0,0,0,1,0,0,0,0,3
4,0.778058,0.001282,0.007885,0.810770,0.001783,NaN,0.005630,0.222387,0.001884,0.019340,...,1,0,0,0,1,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276567,0.252494,0.329432,0.165080,0.028412,0.507827,0.164967,0.006664,NaN,0.754016,0.784514,...,0,0,1,0,1,0,0,0,0,1
276568,0.642230,0.619251,0.036341,1.004678,0.008373,0.116031,0.004885,NaN,0.005760,0.019112,...,0,1,0,0,1,0,0,0,0,1
276569,0.450034,0.032258,0.441105,0.032592,0.504759,0.161592,0.007296,0.174857,0.379909,1.031679,...,0,0,1,0,0,0,0,0,1,1
276570,0.423323,0.650394,0.213818,0.117303,0.006089,0.112029,0.002316,0.014977,0.751193,0.722948,...,1,0,0,0,1,0,0,0,0,1


In [21]:
df_featurizado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276572 entries, 0 to 276571
Data columns (total 138 columns):
 #    Column                     Dtype  
---   ------                     -----  
 0    P_2                        float64
 1    D_39                       float64
 2    B_1                        float64
 3    B_2                        float64
 4    R_1                        float64
 5    S_3                        float64
 6    D_41                       float64
 7    D_43                       float64
 8    D_44                       float64
 9    B_4                        float64
 10   D_45                       float64
 11   B_5                        float64
 12   R_2                        float64
 13   D_46                       float64
 14   D_47                       float64
 15   B_6                        float64
 16   B_7                        float64
 17   D_50                       float64
 18   D_51                       float64
 19   B_9                  